In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import timm
import os
import torchvision.transforms as TT
from tqdm.auto import tqdm
import PIL
from timm.data import create_transform

In [2]:
train = pd.read_parquet('/kaggle/input/vseros-avito-stage1/train_dataset.parquet')
test = pd.read_parquet('/kaggle/input/vseros-avito-stage1/test_dataset.parquet')
sub = pd.read_csv('/kaggle/input/vseros-avito-stage1/sample_submission.csv')

In [3]:
def prepare_dataset(df):
    df = df.copy()
    df[[f'image_{i}' for i in range(4)]] = [[f"{idx}_{i}" for i in range(4)] for idx in df['ID']]
    # df['audiosistema'] = df['audiosistema'].fillna('0').str[0].astype(int)
    # df['owners_count'] = df['owners_count'].replace({'> 3': 4}).astype(int)
    # df['crashes_count'] = df['crashes_count'].replace({'3+': 4}).fillna('-1').astype(int)
    # df['text'] = df.apply(combine_text_features, axis=1)
    return df

train_df = prepare_dataset(train)
test_df = prepare_dataset(test)

In [4]:
img_cols = ['image_0', 'image_1', 'image_2', 'image_3']
train_df[img_cols] = '/kaggle/input/vseros-avito-stage1/AvitoAuto/АвтоПрайс/train_images/' + train_df[img_cols] + '.jpg'
test_df[img_cols] = '/kaggle/input/vseros-avito-stage1/AvitoAuto/АвтоПрайс/test_images/' + test_df[img_cols] + '.jpg'

In [5]:
class AvitoDataset(Dataset):
    def __init__(self, df, data_config, mode = 'train',):
        self.df = df
        self.mode = mode
        if self.mode == 'train':
            self.transforms = timm.data.create_transform(**data_config, is_training=True)
        else:
            self.transforms = timm.data.create_transform(**data_config, is_training=False)

    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        images = []
        for path in self.df[['image_0', 'image_1', 'image_2', 'image_3']].iloc[idx].values:
            if os.path.exists(path):
                with PIL.Image.open(path) as img:
                    images.append(self.transforms(img))
        try:
            y = self.df['price_TARGET'].iloc[idx] // 1000
        except:
            y = 0
        
        return images, y 

In [6]:
class EmbeddingModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.extractor = model
    def forward(self, x):
        embs = []
        for x_i in x:
            embs.append(self.extractor(x_i))
        img_embedding = torch.stack(embs, dim=0).mean(dim=0)
        return img_embedding

In [7]:
tr_embs = []
model = timm.create_model(
            model_name='efficientnet_b3.ra2_in1k',
            pretrained=True,
            num_classes=0
        )
data_config = timm.data.resolve_model_data_config(model)
model = EmbeddingModel(model).to('cuda')
train_dl = DataLoader(AvitoDataset(train_df, data_config, mode='val'), shuffle=False, batch_size=1)

with torch.no_grad():
    model.eval()
    for x, y in tqdm(train_dl):
        x = [x_i.to('cuda') for x_i in x]
        out = model(x)
        tr_embs.append(out.cpu().numpy())

model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

  0%|          | 0/70000 [00:00<?, ?it/s]

In [8]:
tst_embs = []
model = timm.create_model(
            model_name='efficientnet_b3.ra2_in1k',
            pretrained=True,
            num_classes=0
        )
data_config = timm.data.resolve_model_data_config(model)
model = EmbeddingModel(model).to('cuda')
dl = DataLoader(AvitoDataset(test_df, data_config, mode='val'), shuffle=False, batch_size=1)

with torch.no_grad():
    model.eval()
    for x, y in tqdm(dl):
        x = [x_i.to('cuda') for x_i in x]
        out = model(x)
        tst_embs.append(out.cpu().numpy())

  0%|          | 0/25000 [00:00<?, ?it/s]

In [9]:
tr_embeddings = np.stack(tr_embs).squeeze(1)
tst_embeddings = np.stack(tst_embs).squeeze(1)

In [10]:
np.save('train_embeddings_b3.npy', tr_embeddings)
np.save('test_embeddings_b3.npy', tst_embeddings)